In [21]:
# This program predicts stock prices by using machine learning models
# -*- coding: utf-8 -*-
#Install the dependencies

import statistics as stat
import matplotlib as matlib
import bs4 as b4
import sklearn as sklrnb
import datetime as dt
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import pandas_datareader.data as web
import os
import shutil
import bs4 as bs
import pickle
import requests

import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

import yfinance as yf

In [22]:
# Get the list of sp500 tickers
def save_sp500_tickers():
    try:
        f= open("sp500tickers.pickle","rb")
        print ('Using Pickle')
        tickers = pickle.loads(f.read())
        f.close()
    except IOError as err:
        resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        soup = bs.BeautifulSoup(resp.text, 'lxml')
        table = soup.find('table', {'class': 'wikitable sortable'})
        tickers = []
        for row in table.findAll('tr')[1:]:
            ticker = row.findAll('td')[0].text
            ticker = ticker.replace('\r', '')
            ticker = ticker.replace('\n', '')
            ticker = ticker.replace('.', '-')
            tickers.append(ticker)
        with open("sp500tickers.pickle","wb") as f:
            pickle.dump(tickers,f)  
    return tickers


In [23]:
tickers = save_sp500_tickers()

tickers_first = tickers[:150]
tickers_second = tickers[150:300]
tickers_third = tickers[300:]

# print(tickers)
# print(type(tickers))

Using Pickle


In [34]:
def get_predicted_price(ticker):
    try:
        # Get the Adjusted Close Price
        df = yf.Ticker(ticker).history(start='2014-12-3',end='2019-12-03').dropna()
        print(df.head())

        df = df[['Close']]
        #Take a look at the new data
        print(df.head())
        
        # A variable for predicting 'n' days out into the future
        forecast_out = 30 * 3 #'n=30' days
        #Create another column (the target or dependent variable) shifted 'n' units up
        df['Prediction'] = df[['Close']].shift(-forecast_out)
    
        #print the new data set
        print(df.tail())
    
        ### Create the independent data set (X)  #######
        # Convert the dataframe to a numpy array
        X = np.array(df.drop(['Prediction'],1))

        #Remove the last 'n' rows
        X = X[:-forecast_out]
        print(X)
        
        ### Create the dependent data set (y)  #####
        # Convert the dataframe to a numpy array (All of the values including the NaN's)
        y = np.array(df['Prediction'])
        # Get all of the y values except the last 'n' rows
        y = y[:-forecast_out]
        print(y)
    
        # Split the data into 80% training and 20% testing
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    #     # Create and train the Support Vector Machine (Regressor)
    #     svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    #     svr_rbf.fit(x_train, y_train)
    
    #     # Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
    #     # The best possible score is 1.0
    #     svm_confidence = svr_rbf.score(x_test, y_test)
    #     print("svm confidence: ", svm_confidence)
    
        # Create and train the Linear Regression  Model
        lr = LinearRegression()
        # Train the model
        lr.fit(x_train, y_train)
    
        # Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
        # The best possible score is 1.0
        lr_confidence = lr.score(x_test, y_test)
        print("lr confidence: ", lr_confidence)
    
        # Set x_forecast equal to the last 30 rows of the original data set from Close column
        x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
        print(x_forecast)
    
        # Print linear regression model predictions for the next 'n' days
        lr_prediction = lr.predict(x_forecast)
        print(lr_prediction)

    #     # Print support vector regressor model predictions for the next 'n' days
    #     svm_prediction = svr_rbf.predict(x_forecast)
    #     print(svm_prediction)
    except: 
        print(ticker + "has error!")
        continue
    
    return pd.DataFrame(lr_prediction)

SyntaxError: 'continue' not properly in loop (<ipython-input-34-b91c2d5ac1fd>, line 72)

In [37]:
appended_data=[]
for ticker in tickers_third:
    print (ticker)
    df = get_predicted_price(ticker)
    df['ticker'] = ticker
    appended_data.append (df)
    print (df.head())
   
alldf= pd.concat(appended_data)
txtName='sp500Prediction_P3.csv'

alldf.to_csv(txtName)
shutil.copy(txtName, os.path.join(os.environ["HOMEPATH"], "Desktop\stocks"))
print ('done')

LYB
             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2014-12-03  63.43  65.53  62.99  64.35  7430000        0.0             0
2014-12-04  63.97  66.33  63.93  66.33  7154100        0.0             0
2014-12-05  68.20  68.21  65.57  66.03  7066100        0.0             0
2014-12-08  65.55  66.03  62.34  63.52  7485800        0.0             0
2014-12-09  63.20  64.38  62.67  64.33  5000900        0.0             0
            Close
Date             
2014-12-03  64.35
2014-12-04  66.33
2014-12-05  66.03
2014-12-08  63.52
2014-12-09  64.33
            Close  Prediction
Date                         
2019-11-26  92.62         NaN
2019-11-27  93.50         NaN
2019-11-29  92.54         NaN
2019-12-02  92.65         NaN
2019-12-03  91.76         NaN
[[64.35]
 [66.33]
 [66.03]
 ...
 [84.24]
 [83.41]
 [83.25]]
[79.52 79.02 78.8  ... 92.54 92.65 91.76]
lr confidence:  0.3590436037337069
[[81.81]
 [8

             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2014-12-03  49.05  49.39  48.61  49.32  2802000        0.0             0
2014-12-04  49.12  49.12  48.04  48.42  4674500        0.0             0
2014-12-05  48.31  48.72  47.96  48.30  3644400        0.0             0
2014-12-08  48.15  48.25  47.41  47.69  3551200        0.0             0
2014-12-09  46.93  47.57  46.65  47.54  4240600        0.0             0
            Close
Date             
2014-12-03  49.32
2014-12-04  48.42
2014-12-05  48.30
2014-12-08  47.69
2014-12-09  47.54
            Close  Prediction
Date                         
2019-11-26  15.00         NaN
2019-11-27  15.11         NaN
2019-11-29  14.95         NaN
2019-12-02  15.03         NaN
2019-12-03  14.51         NaN
[[49.32]
 [48.42]
 [48.3 ]
 ...
 [22.21]
 [21.75]
 [21.85]]
[53.15 52.96 52.   ... 14.95 15.03 14.51]
lr confidence:  0.5775806736219115
[[21.67]
 [21.52

             Open   High    Low  Close  Volume  Dividends  Stock Splits
Date                                                                   
2014-12-03  62.40  64.69  62.40  64.13  263900        0.0             0
2014-12-04  64.37  65.30  64.25  64.78  205600        0.0             0
2014-12-05  64.75  66.02  64.75  65.48  139000        0.0             0
2014-12-08  65.47  66.82  64.86  65.38  144300        0.0             0
2014-12-09  64.69  66.82  64.00  66.69  136600        0.0             0
            Close
Date             
2014-12-03  64.13
2014-12-04  64.78
2014-12-05  65.48
2014-12-08  65.38
2014-12-09  66.69
             Close  Prediction
Date                          
2019-11-26  401.90         NaN
2019-11-27  406.43         NaN
2019-11-29  403.82         NaN
2019-12-02  398.97         NaN
2019-12-03  397.67         NaN
[[ 64.13]
 [ 64.78]
 [ 65.48]
 ...
 [333.28]
 [343.  ]
 [346.93]]
[ 85.38  86.56  84.17 ... 403.82 398.97 397.67]
lr confidence:  0.8056861243659327
[[33

              Open    High     Low   Close  Volume  Dividends  Stock Splits
Date                                                                       
2014-12-03  112.98  116.55  112.98  114.63  914300        0.0             0
2014-12-04  114.53  115.64  113.93  114.31  434200        0.0             0
2014-12-05  113.97  114.78  112.37  112.76  503700        0.0             0
2014-12-08  112.18  112.56  108.05  108.74  983700        0.0             0
2014-12-09  108.39  109.71  107.55  109.68  701300        0.0             0
             Close
Date              
2014-12-03  114.63
2014-12-04  114.31
2014-12-05  112.76
2014-12-08  108.74
2014-12-09  109.68
             Close  Prediction
Date                          
2019-11-26  268.52         NaN
2019-11-27  271.34         NaN
2019-11-29  268.40         NaN
2019-12-02  268.01         NaN
2019-12-03  270.65         NaN
[[114.63]
 [114.31]
 [112.76]
 ...
 [225.8 ]
 [220.95]
 [222.88]]
[136.27 136.11 134.41 ... 268.4  268.01 270.65]
lr c

             Open   High    Low  Close  Volume  Dividends  Stock Splits
Date                                                                   
2014-12-03  67.08  67.08  66.34  66.65  482300        0.0             0
2014-12-04  66.71  66.96  66.14  66.39  376100        0.0             0
2014-12-05  66.43  66.83  66.19  66.65  383200        0.0             0
2014-12-08  66.78  67.06  66.24  66.59  435500        0.0             0
2014-12-09  66.22  66.73  65.93  66.57  494600        0.0             0
            Close
Date             
2014-12-03  66.65
2014-12-04  66.39
2014-12-05  66.65
2014-12-08  66.59
2014-12-09  66.57
             Close  Prediction
Date                          
2019-11-26  168.93         NaN
2019-11-27  169.43         NaN
2019-11-29  169.25         NaN
2019-12-02  169.21         NaN
2019-12-03  170.84         NaN
[[ 66.65]
 [ 66.39]
 [ 66.65]
 ...
 [158.29]
 [160.04]
 [162.02]]
[ 71.24  71.16  70.23 ... 169.25 169.21 170.84]
lr confidence:  0.87038905381779
[[161.

...

             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2014-12-03  28.68  28.78  28.54  28.72  2227400        0.0             0
2014-12-04  28.66  28.67  28.42  28.62  2150500        0.0             0
2014-12-05  28.57  28.82  28.41  28.80  2673500        0.0             0
2014-12-08  28.82  29.32  28.80  29.27  3769100        0.0             0
2014-12-09  28.93  29.47  28.90  29.31  3630100        0.0             0
            Close
Date             
2014-12-03  28.72
2014-12-04  28.62
2014-12-05  28.80
2014-12-08  29.27
2014-12-09  29.31
            Close  Prediction
Date                         
2019-11-26  29.28         NaN
2019-11-27  29.41         NaN
2019-11-29  29.17         NaN
2019-12-02  28.87         NaN
2019-12-03  28.99         NaN
[[28.72]
 [28.62]
 [28.8 ]
 ...
 [24.92]
 [24.47]
 [25.58]]
[26.17 26.28 25.79 ... 29.17 28.87 28.99]
lr confidence:  0.19389499558974266
[[25.22]
 [25.4

              Open    High     Low   Close  Volume  Dividends  Stock Splits
Date                                                                       
2014-12-03  103.50  105.07  102.73  105.07  194200        0.0           0.0
2014-12-04  105.07  105.73  104.73  105.70  144800        0.0           0.0
2014-12-05  105.92  106.40  105.29  106.21  159200        0.0           0.0
2014-12-08  106.16  107.25  105.58  106.98  151200        0.0           0.0
2014-12-09  105.22  106.47  104.44  106.35  207100        0.0           0.0
             Close
Date              
2014-12-03  105.07
2014-12-04  105.70
2014-12-05  106.21
2014-12-08  106.98
2014-12-09  106.35
             Close  Prediction
Date                          
2019-11-26  194.82         NaN
2019-11-27  196.35         NaN
2019-11-29  196.44         NaN
2019-12-02  195.22         NaN
2019-12-03  192.52         NaN
[[105.07]
 [105.7 ]
 [106.21]
 ...
 [196.53]
 [196.  ]
 [198.53]]
[121.35 120.69 119.71 ... 196.44 195.22 192.52]
lr c

             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2014-12-03  32.74  33.64  32.72  33.61  4057200        0.0           0.0
2014-12-04  33.35  33.71  33.24  33.59  2621700        0.0           0.0
2014-12-05  33.57  33.88  33.47  33.88  1678600        0.0           0.0
2014-12-08  33.97  33.97  33.19  33.26  2282700        0.0           0.0
2014-12-09  32.88  33.12  32.64  33.05  3098500        0.0           0.0
            Close
Date             
2014-12-03  33.61
2014-12-04  33.59
2014-12-05  33.88
2014-12-08  33.26
2014-12-09  33.05
            Close  Prediction
Date                         
2019-11-26  38.30         NaN
2019-11-27  38.80         NaN
2019-11-29  38.93         NaN
2019-12-02  38.50         NaN
2019-12-03  38.25         NaN
[[33.61]
 [33.59]
 [33.88]
 ...
 [34.8 ]
 [34.26]
 [34.15]]
[31.13 30.68 30.61 ... 38.93 38.5  38.25]
lr confidence:  0.155831570758716
[[33.66]
 [32.51]

             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2014-12-03  50.36  50.39  49.37  49.79  4193800        0.0           0.0
2014-12-04  49.70  50.17  49.31  49.85  3092600        0.0           0.0
2014-12-05  50.08  50.36  49.84  50.22  2554400        0.0           0.0
2014-12-08  49.47  49.80  48.87  49.04  4730600        0.0           0.0
2014-12-09  48.49  48.72  48.10  48.24  6214000        0.0           0.0
            Close
Date             
2014-12-03  49.79
2014-12-04  49.85
2014-12-05  50.22
2014-12-08  49.04
2014-12-09  48.24
             Close  Prediction
Date                          
2019-11-26   99.75         NaN
2019-11-27  100.64         NaN
2019-11-29  100.67         NaN
2019-12-02  100.56         NaN
2019-12-03   99.45         NaN
[[ 49.79]
 [ 49.85]
 [ 50.22]
 ...
 [111.77]
 [112.  ]
 [113.12]]
[ 51.83  51.76  51.93 ... 100.67 100.56  99.45]
lr confidence:  0.87894520905422

             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2014-12-03  43.20  43.46  42.86  43.33  2003600        0.0             0
2014-12-04  43.25  43.35  42.34  42.48  4294300        0.0             0
2014-12-05  42.45  42.93  42.32  42.46  3314000        0.0             0
2014-12-08  42.68  42.86  42.36  42.58  5889300        0.0             0
2014-12-09  41.69  42.96  41.27  42.31  5515000        0.0             0
            Close
Date             
2014-12-03  43.33
2014-12-04  42.48
2014-12-05  42.46
2014-12-08  42.58
2014-12-09  42.31
             Close  Prediction
Date                          
2019-11-26  121.89         NaN
2019-11-27  121.63         NaN
2019-11-29  120.52         NaN
2019-12-02  119.91         NaN
2019-12-03  120.53         NaN
[[ 43.33]
 [ 42.48]
 [ 42.46]
 ...
 [115.29]
 [114.5 ]
 [114.96]]
[ 45.55  45.68  45.32 ... 120.52 119.91 120.53]
lr confidence:  0.93572280307153